### Volumes

<b>O que são volumes?</b>

- Uma forma prática de persistir dados em aplicações e não depender de containers para isso;

- Todo dado criado por um container é salvo nele, quando o container é removido perdemos os dados;

- Então precisamos dos volumes para gerenciar os dados e também conseguir fazer backups de forma mais simples;

<b>Tipos de volumes</b>

- ```Anônimos (anonymous)```: Diretórios criados pela ```flag -v```, porém com um nome aleatório;

- ```Nomeados (named)```: São volumes com nomes, podemos nos referir a estes facilmente e saber para que são utilizados no nosso ambiente;

- ```Bind mounts```: Uma forma de salvar dados na nossa máquina, sem o gerenciamento do Docker, informamos um diretório para este fim;


<b>O problema da persistência</b>

- Se criarmos um container com alguma imagem, todos os arquivos que geramos dentro dele serão do container;

- Quando o container for removido, perderemos estes arquivos;

- Por isso precisamos os volumes;

- Vamos criar um exemplo prático!

```Dockerfile
FROM php:8-apache

WORKDIR /var/www/html/

COPY . .

EXPOSE 80

RUN chown -R www-data:www-data /var/www 

```

In [ ]:
! docker build -t bruiglesias/php-image .

In [ ]:
! docker run -d -p 80:80 --name php-container bruiglesias/php-image

<b>Volumes anônimos</b>

- Podemos criar um volume ```anônimo (anonymous```) da seguinte maneira: ```docker run -v /data```

- Onde ```/data``` será o diretório que contém o volume anônimo;

- E este container estará atrelado ao volume anônimo;

- Com o comando ```docker volume ls```, podemos ver todos os volumes do nosso ambiente;


In [1]:
! docker stop php-container

php-container


In [2]:
! docker container rm php-container

php-container


In [3]:
! docker run -d -p 80:80 --name php-container --rm -v /data bruiglesias/php-image

df3494005fd64c72b4a1400366789ebc8d1a128a787d6d498ed6a44431551982


In [4]:
! docker volume ls

DRIVER    VOLUME NAME
local     8b1f7c23ec2b050e880af239eff367b14185bf7ed10f47ec8acafb403007bba9


<b>Volumes nomeados</b>

- Podemos criar um volume ```nomeado (named)``` da seguinte maneira: ```docker run -v nomedovolume:/data```

- Agora o volume tem um nome e pode ser facilmente referenciado;

- Em ```docker volume ls``` podemos verificar o container nomeado criado;

- Da mesma maneira que o anônimo, este volume tem como função armazenar arquivos;

In [5]:
! docker stop php-container

php-container


In [6]:
! docker run -d -p 80:80 --name php-container --rm -v volume-php:/var/www/html/messages  bruiglesias/php-image

11429cc00a54f9d0da047b5c945fdbbceabe5e88fa239a070ffad5bec15f7e35


In [7]:
! docker volume ls

DRIVER    VOLUME NAME
local     volume-php


In [8]:
! docker stop php-container

php-container


<b>Bind mounts</b>

- Bind mount também é um volume, porém ele fica em um diretório que nós especificamos;

- Então não criamos um volume em sim, apontamos um diretório;

- O comando para criar um bind mount é: ```docker run /dir/data:/data```

- Desta maneira o diretório ```/dir/data``` no nosso computador, será o volume deste container;

In [ ]:
! docker run -d -p 80:80 --user $(id -u):$(id -g) --name php-container --rm -v /home/iglesias/Documentos/curso_docker/volumes/messages:/var/www/html/messages bruiglesias/php-image

b00b46fbdba2951c3682d5ca5c18ea006f795d5b91a287526e4416d49d1361d3


In [22]:
! docker stop php-container

php-container



<b>Atualização do projeto com bind mount</b>

- Bind mount não serve apenas para volumes!

- Podemos utilizar esta técnica para ```atualização em tempo real do projeto```;

- Sem ter que refazer o build a cada atualização do mesmo;

- Vamos ver na prática!

In [16]:
!docker run -d -p 80:80 --user $(id -u):$(id -g) --name php-container --rm -v /home/iglesias/Documentos/curso_docker/volumes:/var/www/html bruiglesias/php-image

10af2961753d8b3d39bfa50ea99f3dfc48068b85c0116601a544738cf01c1b36


<b>Criar um volume</b>

- Podemos criar volumes manualmente também;

- Utilizamos o comando: ```docker volume create <nome>```;

- Desta maneira temos um named volume criado, podemos atrelar a algum container na execução do mesmo;

In [17]:
! docker volume create volume-manual

volume-manual


In [19]:
! docker volume ls

DRIVER    VOLUME NAME
local     volume-manual
local     volume-php


<b>Listando todos os volumes</b>

- Com o comando: ```docker volume ls``` listamos todos todos os volumes;

- Desta maneira temos acesso aos ```anonymous``` e os ```named volumes```;

- Interessante para saber quais volumes estão criados no nosso ambiente;

In [20]:
! docker volume ls

DRIVER    VOLUME NAME
local     volume-manual
local     volume-php


<b>Checar um volume</b>

- Podemos verificar os detalhes de um volume em específico com o comando: ```docker volume inspect nome```;

- Desta forma temos acesso ao local em que o volume guarda dados, nome, escopo e muito mais;

- O docker salva os dados dos volumes em algum diretório do nosso computador, desta forma podemos saber qual é;

In [21]:
! docker volume inspect volume-php

[
    {
        "CreatedAt": "2024-11-24T20:56:51-04:00",
        "Driver": "local",
        "Labels": null,
        "Mountpoint": "/var/lib/docker/volumes/volume-php/_data",
        "Name": "volume-php",
        "Options": null,
        "Scope": "local"
    }
]


<b>Remover um volume</b>

- Podemos também remover um volume existente de forma fácil;

- Vamos utilizar o comando ```docker volume rm <nome>```

- Observe que os dados serão removidos todos também, tome cuidado com este comando;

In [23]:
! docker volume rm volume-manual

volume-manual


<b>Removendo volumes não utilizados</b>

- Podemos remover todos os volumes que não estão sendo utilizados com apenas um comando;

- O comando é: ```docker volume prune```

- Semelhante ao prune que remove imagens e containers, visto anteriormente;

In [ ]:
! docker volume prune

<b>Volume apenas de leitura</b>

- Podemos criar um volume que tem apenas permissão de leitura, isso é útil em algumas aplicações;

- Para realizar esta configuração devemos utilizar o comando: docker ```run -v volume:/data:ro```

- Este ```:ro``` é a abreviação de read only;